Codes to simulate the Mutual Repression Model represented by the Delay Differential Equations.

In [ ]:
pip install ddeint

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import pylab as p

In [ ]:
# Code to simulate the delay differential equations

from ddeint import ddeint
def model(Y,t,d,μ,n,b,θ):
    x,y = Y(t)
    xd,yd = Y(t-d)
    return np.array([-μ*x + (1+b)*θ**n/(θ**n+yd**n), -μ*y + θ**n/(θ**n+xd**n)])

g = lambda t : np.array([0,0])
ts = np.linspace(0,40,20000)

d=2
yy = ddeint(model,g,ts,fargs=(d,1,3,-0.05,0.25))

plt.plot(yy[:,0], yy[:,1], lw=2)
plt.xlabel("X"); plt.ylabel("Y");
plt.legend()
plt.show

In [ ]:
#Plotting the time coarse of X and Y
plt.plot(ts,abs(yy[:,0]), lw=2, label="X")
plt.plot(ts,abs(yy[:,1]), lw=2, label="Y")
plt.xlabel('Time')
plt.ylabel('Levels of X and Y')
plt.legend()

In [ ]:
from ddeint import ddeint
import numpy as np
import matplotlib.pyplot as plt

# Define the model function representing the time delay differential equation (DDE) system
def model(Y, t, d, μ, n, b, θ):
    x, y = Y(t)  # Current state variables x and y
    xd, yd = Y(t - d)  # Delayed state variables xd and yd
    return np.array([-μ*x + (1+b)*θ**n/(θ**n+yd**n), -μ*y + θ**n/(θ**n+xd**n)])

g = lambda t: np.array([0, 0])  # Define the history function representing the initial condition

ts = np.linspace(0, 300, 20000)  # Time points at which the solution will be computed

# Range of delay values
delay_values = np.linspace(0, 3.5, 30)

# Empty lists to store delay values, corresponding steady state values, and switch times
delay_list = []
steady_state_list = []
switch_time_list = []

# Iterate over delay values
for d in delay_values:
    # Solve the DDE system using ddeint
    yy = ddeint(model, g, ts, fargs=(d, 1, 3, -0.005, 0.25))
    
    # Get the steady state values
    steady_state = (yy[-1, 0], yy[-1, 1])
    
    # Append the delay and steady state values to the respective lists
    delay_list.append(d)
    steady_state_list.append(steady_state)
    
    # Find the first time the solution reaches the steady state
    diff = np.linalg.norm(yy - steady_state, axis=1)
    switch_time = ts[np.argmax(diff < 0.05)]
    switch_time_list.append(switch_time)

    
    
    
# Plotting the relationship between delay values and time to switch
plt.plot(delay_list, switch_time_list, lw=2)
plt.xlabel("Delay Values(τ)")
plt.ylabel("Time to Switch")

plt.grid(True)
plt.show()

In [ ]:
# Code to Calculate the unstable steady state.


import numpy as np
from scipy.optimize import root

def zero_curve(XY, n):
    x, y = XY[0], XY[1]
    fX = -mu * x + (1 + b) * (theta**n / (theta**n + y**n))
    fY = -mu * y + (theta**n / (theta**n + x**n))
    return [fX, fY]

# Parameter values
mu = 1
theta = 0.25
b = -0.005

# Values of n
n_values = np.linspace(1.7, 5,20 )

diff_list = []
solx = []
soly = []
    
# Find the fixed points for each value of n
fixed_points = []
for n in n_values:
    # Initial guess for the fixed point
    initial_guess = [0, 0]
    
    # Solve for the fixed point using scipy.optimize.root
    sol = root(zero_curve, initial_guess, args=(n,))
    
    # Check if the solver successfully converged to a solution
    if sol.success:
        fixed_points.append(sol.x)
        diff = np.abs(sol.x[0] - sol.x[1])
        solx.append(sol.x[0])
        soly.append(sol.x[1])
        print(f"Fixed point for n={n}: (X, Y) = ({sol.x[0]:.4f}, {sol.x[1]:.4f}), |X - Y| = {diff:.4f}")
    else:
        print(f"No fixed point found for n={n}")
    
    diff_list.append(diff)
    diff_dict = dict(zip(n_values,diff_list))
    solx_dict = dict(zip(n_values,solx))
    soly_dict = dict(zip(n_values,soly))

In [ ]:
# Code to calculate the sensitivity


def model(Y, t, d, μ, n, b, θ):
    x, y = Y(t)  
    xd, yd = Y(t - d)  
    return np.array([-μ*x + (1+b)*θ**n/(θ**n+yd**n), -μ*y + θ**n/(θ**n+xd**n)])

g = lambda t: np.array([0, 0])  # Define the history function 

ts = np.linspace(0, 300, 20000)  # Time points 


mu = 1
theta = 0.25
b = -0.005


n_values = np.linspace(1.7, 5, 20)
diff_dict = dict(zip(n_values,diff_list))
# Empty lists
n_list = []
switch_time_list = []
sensitivity_values = []
th = []

for n in n_values:
    ki = n * mu ** 2 * ((solx_dict[n] * soly_dict[n]) ** ((n + 1) / 2)) / (theta ** n * (1 + b))
    sensitivity_values.append(ki)
    
    yy = ddeint(model, g, ts, fargs=(2, 1, n, -0.005, 0.25))
    
    stable_state = (yy[-1, 0], yy[-1, 1])
    
    normal = np.linalg.norm(yy - stable_state, axis=1)
    threshold = 0.95*diff_dict[n]
    th.append(threshold)
    
    switch_time = ts[np.argmax( normal < threshold)]
    switch_time_list.append(switch_time)
    
print(switch_time_list)

#plt.subplot(1, 2, 1)

plt.plot(sensitivity_values, switch_time_list, lw=2, label='Switch Time')
plt.xlabel("Sensitivity")
plt.ylabel("Time to Switch")
plt.title(" Sensitivity vrs Time to Switch")
plt.legend()
plt.grid(True)
plt.show()



